In [1]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import os

In [2]:
epochs = 100
learning_rate = 0.001
decay_rate = learning_rate / epochs

In [3]:
model = Sequential()

# 1st Convolutional Layer
model.add(Conv2D(filters=96, input_shape=(227,227,3), kernel_size=(11,11), 
                 strides=(4,4), padding='valid'))
model.add(Activation('relu'))
# Pooling 
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))
# Batch Normalisation before passing it to the next layer
model.add(BatchNormalization())

# 2nd Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), padding='same'))
model.add(Activation('relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))
# Batch Normalisation
model.add(BatchNormalization())

# 3rd Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
model.add(Activation('relu'))
# Batch Normalisation
model.add(BatchNormalization())

# 4th Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
model.add(Activation('relu'))
# Batch Normalisation
model.add(BatchNormalization())

# 5th Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same'))
model.add(Activation('relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))
# Batch Normalisation
model.add(BatchNormalization())

# Passing it to a dense layer
model.add(Flatten())
# 1st Dense Layer
model.add(Dense(4096, input_shape=(227*227*3,)))
model.add(Activation('relu'))
# Add Dropout to prevent overfitting
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())

# 2nd Dense Layer
model.add(Dense(4096))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())

# 3rd Dense Layer
model.add(Dense(1000))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())

# Output Layer
model.add(Dense(2))
model.add(Activation('softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 55, 55, 96)        34944     
_________________________________________________________________
activation (Activation)      (None, 55, 55, 96)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 27, 27, 96)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 27, 27, 96)        384       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 27, 27, 256)       614656    
_________________________________________________________________
activation_1 (Activation)    (None, 27, 27, 256)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 256)       0

In [4]:
test_datagen = ImageDataGenerator(rescale = 1./255) 
train_datagen = ImageDataGenerator(rescale = 1./255,
                                 validation_split=0.20)
training_set = train_datagen.flow_from_directory('dataset\\traning_set\\',
                                                target_size = (227,227),
                                                batch_size = 32,
                                                class_mode = 'categorical',
                                                color_mode="rgb",
                                                subset='training',
                                                shuffle=True,
                                                seed=42)
valid_set = train_datagen.flow_from_directory('dataset\\traning_set\\',
                                                target_size=(227,227),
                                                color_mode="rgb",
                                                batch_size=32,
                                                class_mode="categorical",
                                                subset='validation',
                                                shuffle=True,
                                                seed=42)


test_set = test_datagen.flow_from_directory('dataset\\test_set\\',
                                                target_size = (227,227),
                                                batch_size = 1,
                                                color_mode="rgb",
                                                shuffle=False,
                                                class_mode = None,
                                                seed=42)


Found 2168 images belonging to 2 classes.
Found 541 images belonging to 2 classes.
Found 314 images belonging to 2 classes.


In [5]:
print(training_set)

In [6]:
model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.SGD(lr=learning_rate, momentum=0.9, decay=decay_rate, nesterov=False), metrics = ['accuracy'])

from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint("best_model_1.hdf5", monitor='val_accuracy', verbose=1,
    save_best_only=True, mode='max', save_freq="epoch")

history = model.fit(training_set,epochs = epochs,validation_data = valid_set, callbacks = [checkpoint])

Epoch 1/100
68/68 [==============================] - 660s 10s/step - loss: 0.8471 - accuracy: 0.5616 - val_loss: 0.7225 - val_accuracy: 0.5139

Epoch 00001: val_accuracy improved from -inf to 0.51386, saving model to best_model_1.hdf5
Epoch 2/100
68/68 [==============================] - 501s 7s/step - loss: 0.7707 - accuracy: 0.6225 - val_loss: 0.7583 - val_accuracy: 0.5139

Epoch 00002: val_accuracy did not improve from 0.51386
Epoch 3/100
68/68 [==============================] - 474s 7s/step - loss: 0.7193 - accuracy: 0.6342 - val_loss: 0.7157 - val_accuracy: 0.5545

Epoch 00003: val_accuracy improved from 0.51386 to 0.55453, saving model to best_model_1.hdf5
Epoch 4/100
68/68 [==============================] - 519s 8s/step - loss: 0.6896 - accuracy: 0.6555 - val_loss: 0.7818 - val_accuracy: 0.5250

Epoch 00004: val_accuracy did not improve from 0.55453
Epoch 5/100
68/68 [==============================] - 524s 8s/step - loss: 0.6454 - accuracy: 0.7001 - val_loss: 0.7212 - val_accurac

KeyboardInterrupt: 

In [7]:
model.save_weights("best_weight1.h5")


In [10]:
print(history)


NameError: name 'history' is not defined

In [8]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
#Train and validation accuracy
plt.plot(epochs, acc, 'b', label='Training accurarcy')
plt.plot(epochs, val_acc, 'r', label='Validation accurarcy')
plt.title('Training and Validation accurarcy')
plt.legend()

plt.figure()
#Train and validation loss
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and Validation loss')
plt.legend()
plt.show()

NameError: name 'history' is not defined

In [12]:
import numpy as np
categories = ["Cancerous","Non cancerous"]
from keras.preprocessing import image

test_image = image.load_img('dataset\\test_set\\cancerous\\C_1404.jpeg',
                            target_size = (227,227))

test_image = image.img_to_array(test_image)
test_image = np.array([test_image], dtype=np.float16) / 255.0
result = new_model.predict(test_image)

print(list(result[0]))
#print('[0.1,0.2,0.5,0.6,0.8,0.1,0.3,0.4]')
print(categories[np.argmax(result)])

[0.99990916, 9.084665e-05]
Cancerous


In [ ]:
predict = model.predict(test_set)

In [ ]:
y_classes = predict.argmax(axis=-1)

In [2]:
from keras.models import load_model
new_model = load_model('best_model_1.hdf5')
#new_model.evaluate(valid_set)

In [35]:
STEP_SIZE_TEST=test_set.n//test_set.batch_size
test_set.reset()
pred=new_model.predict(test_set,
steps=STEP_SIZE_TEST,
verbose=1)

314/314 [==============================] - 24s 76ms/step


In [36]:
predicted_class_indices=np.argmax(pred,axis=1)


In [56]:
labels = (training_set.class_indices)
print(labels)
labels = dict((v,k) for k,v in labels.items())
print(labels)
predictions = [labels[k] for k in predicted_class_indices]

{'cancerous': 0, 'non_cancerous': 1}
{0: 'cancerous', 1: 'non_cancerous'}


In [58]:
import pandas as pd
filenames=test_set.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})
print(results)
results.to_csv("results.csv",index=False)

                       Filename    Predictions
0         cancerous\C_1398.jpeg      cancerous
1         cancerous\C_1399.jpeg      cancerous
2         cancerous\C_1400.jpeg      cancerous
3         cancerous\C_1401.jpeg      cancerous
4         cancerous\C_1402.jpeg      cancerous
..                          ...            ...
309  non_cancerous\NC_1469.jpeg  non_cancerous
310  non_cancerous\NC_1470.jpeg  non_cancerous
311  non_cancerous\NC_1471.jpeg  non_cancerous
312  non_cancerous\NC_1472.jpeg  non_cancerous
313  non_cancerous\NC_1473.jpeg  non_cancerous

[314 rows x 2 columns]


In [1]:
!pip install streamlit



  Created wheel for blinker: filename=blinker-1.4-py3-none-any.whl size=13454 sha256=a22ba12fb300180ed3179384d616465bc34b028661c3934a48578c6bf0524244
  Stored in directory: c:\users\dev-laptop\appdata\local\pip\cache\wheels\b7\a5\68\fe632054a5eadd531c7a49d740c50eb6adfbeca822b4eab8d4
Successfully built blinker
